# Importer les librairies, et fichiers .py auxiliaires

In [1]:
import numpy as np
from preprocess_data import *
from bayes_classifier import BayesClassifier
from complement_naive_bayes import train_cnb_with_tfidf, train_cnb
from ensemble_learning import train_ensemble
from xgboost_classifier import train_xgboost, train_xgboost_with_tfidf
from svc_classifier import *

from save_output import save_output

In [2]:
data_preprocess = DataPreprocess()
X_train, y_train, X_test = data_preprocess.train, data_preprocess.label_train, data_preprocess.test

# Exploration de données

# Jalon 1) Naive de Bayes vanille
K-Fold Validation croisée ($k = 7)$:
- α: Lissage du postérieur de Bayes

In [3]:
bayes_classifier = BayesClassifier()
best_lps, best_f1 = bayes_classifier.hyperparameter_tuning(X_train, y_train)
print(best_lps, best_f1)
tuned_bayes_classifier = BayesClassifier()
tuned_bayes_classifier.fit(X_train, y_train, best_lps)
predictions = np.array([tuned_bayes_classifier.predict(x_i) for x_i in X_test])
save_output(predictions, "bayes", best_lps, "stopwords")

100%|██████████| 13/13 [00:29<00:00,  2.24s/it]


0.9499999999999998 0.5965643817273643
predictions saved in output/20241111/155507_bayes_0.9499999999999998_stopwords.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 None
Number of 0: 1658
Number of 1: 698
Ratio of 1: 0.29626485568760613
Number of differences with bayes classifier submission: 3
Ratio of 1 in bayes submission: 0.29499151103565363


# Jalon 2)
# Naive Bayes Complement
K-Fold validation croisée stratifiée ($k = 5)$:
- α: Lissage du postérieur de Bayes

Prétraitement: TFIDF

In [4]:
complement_naive_bayes, tfidf_transformer = train_cnb_with_tfidf(X_train, y_train)
tfidf_test = tfidf_transformer.transform(X_test)
predictions = complement_naive_bayes.predict(tfidf_test)
save_output(predictions, "cnb", "random_search_50_iter", "tfidf")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END .....cnb__alpha=0.5619101782710437;, score=0.111 total time=   1.4s
[CV 2/2] END .....cnb__alpha=0.5619101782710437;, score=0.110 total time=   1.4s
Best Parameters: {'cnb__alpha': 0.5619101782710437}
Best F1 Score: 0.11089093184257118
predictions saved in output/20241111/155512_cnb_random_search_50_iter_tfidf.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 output/20241111/154325_cnb_random_search_50_iter_tfidf.csv
	 output/20241111/154450_cnb_random_search_50_iter_tfidf.csv
	 output/20241111/154831_cnb_random_search_50_iter_tfidf.csv
	 output/20241110/170114_cnb_random_search_50_iter_tfidf.csv
	 output/20241110/165637_cnb_random_search_50_iter_tfidf.csv
	 output/20241110/170636_cnb_random_search_50_iter_tfidf.csv
	 output/20241110/170853_cnb_random_search_50_iter_tfidf.csv
	 output/20241110/170443_cnb_random_search_50_iter_tfidf.csv
	 output/20241110/170353_cnb_r

Prétraitement: Sans prétraitement

In [5]:
complement_naive_bayes = train_cnb(X_train, y_train)
predictions = complement_naive_bayes.predict(X_test)
save_output(predictions, "cnb", "random_search_50_iter", "no_preprocessing")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END .....cnb__alpha=0.5619101782710437;, score=0.588 total time=   0.5s
[CV 2/2] END .....cnb__alpha=0.5619101782710437;, score=0.575 total time=   0.5s
Best Parameters: {'cnb__alpha': 0.5619101782710437}
Best F1 Score: 0.581482890607508
predictions saved in output/20241111/155514_cnb_random_search_50_iter_no_preprocessing.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 output/20241111/154833_cnb_random_search_50_iter_no_preprocessing.csv
	 output/20241111/154327_cnb_random_search_50_iter_no_preprocessing.csv
	 output/20241111/154452_cnb_random_search_50_iter_no_preprocessing.csv
	 output/20241110/172138_cnb_random_search_50_iter_no_preprocessing.csv
Number of 0: 1544
Number of 1: 812
Ratio of 1: 0.34465195246179964
Number of differences with bayes classifier submission: 117
Ratio of 1 in bayes submission: 0.29499151103565363


Prétraitement: TFIDF, réduction par arbre, stopwords retirés

In [6]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train, sorted_indeces_features = tree_based_dimensionality_reduction(data_preprocess.train, data_preprocess.label_train)
X_test = data_preprocess.test[:, sorted_indeces_features]
complement_naive_bayes = train_cnb(X_train, data_preprocess.label_train)
predictions = complement_naive_bayes.predict(X_test)
save_output(predictions, "cnb", "random_search_50_iter", "tree_reduction_stopwords_tfidf")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END .....cnb__alpha=0.5619101782710437;, score=0.559 total time=   0.0s
[CV 2/2] END .....cnb__alpha=0.5619101782710437;, score=0.555 total time=   0.0s
Best Parameters: {'cnb__alpha': 0.5619101782710437}
Best F1 Score: 0.5571885009594754
predictions saved in output/20241111/155527_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 output/20241111/154846_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf.csv
Number of 0: 1505
Number of 1: 851
Ratio of 1: 0.36120543293718166
Number of differences with bayes classifier submission: 320
Ratio of 1 in bayes submission: 0.29499151103565363


# SVC
Grid Search, K-Fold validation croisée stratifiée ($k = 5)$, avec hyperparamètres:
- $\gamma$ : Coefficient du Noyau RBF
- $C$ : Terme de régularization pour la pénalité euclidienne.

Prétraitement: Retrirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [9]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
indeces_to_remove = remove_cum_sum(data_preprocess.train, 0.95)
data_preprocess.train = np.delete(data_preprocess.train, indeces_to_remove, axis=1)
data_preprocess.test = np.delete(data_preprocess.test, indeces_to_remove, axis=1)
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

best_params_, best_score_ = train_svc(X_train_undersampled, y_train_undersampled)
print(best_params_, best_score_)
svc = SVC(kernel='rbf', C=best_params_['C'], gamma=best_params_['gamma'])
svc.fit(X_train_undersampled, y_train_undersampled)
y_pred = svc.predict(data_preprocess.test)
params = f"C={best_params_['C']}, gamma={best_params_['gamma']}"
save_output(y_pred, "svm", params, "stopwords_cum-sum_undersampled")

{'gamma': [0.003], 'C': [45]}
Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV 1/1] END .................C=45, gamma=0.003;, score=0.635 total time=  51.1s
{'C': 45, 'gamma': 0.003} 0.6349442379182156
predictions saved in output/20241111/160700_svm_C=45, gamma=0.003_stopwords_cum-sum_undersampled.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 None
Number of 0: 1615
Number of 1: 741
Ratio of 1: 0.31451612903225806
Number of differences with bayes classifier submission: 426
Ratio of 1 in bayes submission: 0.29499151103565363


# SGDClassifier

# XGBoost
Random Search, K-Fold validation croisée stratifiée ($k = 5)$, avec hyperparamètres:
- Learning rate: Taille du pas lors de la descente de critère sur la perte (gain de subdivision) 
- Nombre d'estimateurs (n_estimators): Nombre d'arbres de décision
- Profondeur maximale (max_depth): Profondeur maximale de chaque arbre (nombre de branchements max)
- sous-échantillon (subsample):  Proportion de données utilisées pour produire chaque arbre.

Prétraitement: Sur-échantillonnage bootstrap

In [6]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
x_train_oversampled, y_train_oversampled = boostrap_oversampling(data_preprocess.train, data_preprocess.label_train)
xgboost_classifier, tfidf_transformer = train_xgboost(x_train_oversampled, y_train_oversampled)
predictions = xgboost_classifier.predict(data_preprocess.test)
save_output(predictions, "xgboost", "random_search_15_iter", "stopwords_bootstrap")

Fitting 2 folds for each of 1 candidates, totalling 2 fits


ValueError: Invalid parameter 'model' for estimator Pipeline(steps=[('xgboost',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy

Prétraitement: 

# Apprentissage par Ensemble

Ensmeble: CNB, XGBoost, Logistic Regression sans stopwords, sous-échantillonage

Prétraitement : sans stopwords, sous-échantillonage, réduction de dimension par somme cumulative

In [4]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
indeces_to_remove =remove_cum_sum(data_preprocess.train, 0.95)
data_preprocess.train = np.delete(data_preprocess.train, indeces_to_remove, axis=1)
data_preprocess.test = np.delete(data_preprocess.test, indeces_to_remove, axis=1)
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)
model_names = ['ComplementNB', 'XGBoost', 'LogisticRegression']
best_ensemble_model = train_ensemble(X_train_undersampled, y_train_undersampled, model_names)
predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_logreg", "random_search_15_iter", "stopwords_undersampling_cumulative_sum")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END ...model__alpha=0.3851694527491273;, score=0.694 total time=   0.1s
[CV 2/2] END ...model__alpha=0.3851694527491273;, score=0.685 total time=   0.1s
Best F1 for ComplementNB: 0.6895566521985158
Confusion Matrix - Fold 1:
[[1329  452]
 [ 298  851]]

Confusion Matrix - Fold 2:
[[1279  502]
 [ 289  860]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:52:07] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 1/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.630 total time=   1.7s


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:52:09] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 2/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.635 total time=   1.6s


...

Best F1 for XGBClassifier: 0.6321269577091664


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:52:13] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:52:15] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion Matrix - Fold 1:
[[1500  281]
 [ 492  657]]

Confusion Matrix - Fold 2:
[[1452  329]
 [ 462  687]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END model__C=5.588135039273247, model__penalty=l1, model__solver=liblinear;, score=0.581 total time=   0.1s
[CV 2/2] END model__C=5.588135039273247, model__penalty=l1, model__solver=liblinear;, score=0.600 total time=   0.1s
Best F1 for LogisticRegression: 0.5907665275877441
Confusion Matrix - Fold 1:
[[1332  449]
 [ 494  655]]

Confusion Matrix - Fold 2:
[[1304  477]
 [ 451  698]]



/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:52:17] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


predictions saved in output/20241111/155220_ensemble_cnb_xgboost_logreg_random_search_15_iter_stopwords_undersampling_cumulative_sum.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 None
Number of 0: 1569
Number of 1: 787
Ratio of 1: 0.3340407470288625
Number of differences with bayes classifier submission: 250
Ratio of 1 in bayes submission: 0.29499151103565363


Prétraitement : sans stopwords, sous-échantillonage

In [5]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)
model_names = ['ComplementNB', 'XGBoost', 'LogisticRegression']
best_ensemble_model = train_ensemble(X_train_undersampled, y_train_undersampled, model_names)
predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_logreg", "random_search_15_iter", "stopwords_undersampling_cumulative_sum")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END ...model__alpha=0.3851694527491273;, score=0.687 total time=   0.4s
[CV 2/2] END ...model__alpha=0.3851694527491273;, score=0.683 total time=   0.4s
Best F1 for ComplementNB: 0.6849461126344383
Confusion Matrix - Fold 1:
[[1298  483]
 [ 296  853]]

Confusion Matrix - Fold 2:
[[1275  506]
 [ 290  859]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:52:25] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 1/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.630 total time=   4.2s


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:52:29] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 2/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.635 total time=   4.1s


...

Best F1 for XGBClassifier: 0.6321269577091664


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:52:40] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:52:44] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion Matrix - Fold 1:
[[1500  281]
 [ 492  657]]

Confusion Matrix - Fold 2:
[[1452  329]
 [ 462  687]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END model__C=5.588135039273247, model__penalty=l1, model__solver=liblinear;, score=0.587 total time=   0.3s
[CV 2/2] END model__C=5.588135039273247, model__penalty=l1, model__solver=liblinear;, score=0.604 total time=   0.3s
Best F1 for LogisticRegression: 0.5959325131042796
Confusion Matrix - Fold 1:
[[1350  431]
 [ 492  657]]

Confusion Matrix - Fold 2:
[[1306  475]
 [ 447  702]]



/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:52:51] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


predictions saved in output/20241111/155258_ensemble_cnb_xgboost_logreg_random_search_15_iter_stopwords_undersampling_cumulative_sum.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 None
Number of 0: 1569
Number of 1: 787
Ratio of 1: 0.3340407470288625
Number of differences with bayes classifier submission: 236
Ratio of 1 in bayes submission: 0.29499151103565363
